## Select the best model from the grid and create the stacked ensembles


In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


import h2o
h2o.init(min_mem_size='25G')

DATA_LOCATION = "../../data/"
MODELS_LOCATION = "../../models/ALL_FEATURES/"

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 16 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,5 months and 6 days !!!
H2O_cluster_name:,H2O_from_python_azureuser_msv0eq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,24.04 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
train = h2o.import_file( DATA_LOCATION + "processed/final.train.tsv")
train.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


SampleID,NC000962_3.22,NC000962_3.434,NC000962_3.524,NC000962_3.645,NC000962_3.648,NC000962_3.654,NC000962_3.666,NC000962_3.675,NC000962_3.678,NC000962_3.693,NC000962_3.698,NC000962_3.699,NC000962_3.702,NC000962_3.705,NC000962_3.708,NC000962_3.717,NC000962_3.729,NC000962_3.741,NC000962_3.744,NC000962_3.747,NC000962_3.750,NC000962_3.756,NC000962_3.770,NC000962_3.777,NC000962_3.780,NC000962_3.783,NC000962_3.793,NC000962_3.795,NC000962_3.799,NC000962_3.801,NC000962_3.805,NC000962_3.822,NC000962_3.840,NC000962_3.846,NC000962_3.849,NC000962_3.852,NC000962_3.1045,NC000962_3.1049,NC000962_3.1089,NC000962_3.1123,NC000962_3.1152,NC000962_3.1153,NC000962_3.1155,NC000962_3.1161,NC000962_3.1164,NC000962_3.1166,NC000962_3.1167,NC000962_3.1176,NC000962_3.1206,NC000962_3.1212,NC000962_3.1255,NC000962_3.1278,NC000962_3.1291,NC000962_3.1302,NC000962_3.1326,NC000962_3.1389,NC000962_3.1399,NC000962_3.1413,NC000962_3.1416,NC000962_3.1422,NC000962_3.1429,NC000962_3.1431,NC000962_3.1432,NC000962_3.1452,NC000962_3.1458,NC000962_3.1461,NC000962_3.1470,NC000962_3.1473,NC000962_3.1474,NC000962_3.1653,NC000962_3.1676,NC000962_3.1699,NC000962_3.1703,NC000962_3.1708,NC000962_3.1718,NC000962_3.1729,NC000962_3.1771,NC000962_3.1827,NC000962_3.1849,NC000962_3.1918,NC000962_3.1977,NC000962_3.2532,NC000962_3.2745,NC000962_3.3352,NC000962_3.3446,NC000962_3.4013,NC000962_3.4086,NC000962_3.4096,NC000962_3.4119,NC000962_3.4938,NC000962_3.5075,NC000962_3.5627,NC000962_3.5782,NC000962_3.5790,NC000962_3.5791,NC000962_3.5803,NC000962_3.5807,NC000962_3.5824,NC000962_3.5839,NC000962_3.5848,NC000962_3.5856,NC000962_3.5858,NC000962_3.5860,NC000962_3.5902,NC000962_3.6003,NC000962_3.6013,NC000962_3.6112,NC000962_3.6280,NC000962_3.6286,NC000962_3.6292,NC000962_3.6307,NC000962_3.6362,NC000962_3.6382,NC000962_3.6388,NC000962_3.6403,NC000962_3.6430,NC000962_3.6436,NC000962_3.6439,NC000962_3.6445,NC000962_3.6452,NC000962_3.6453,NC000962_3.6502,NC000962_3.6508,NC000962_3.6511,NC000962_3.6515,NC000962_3.6520,NC000962_3.6535,NC000962_3.6547,NC000962_3.6550,NC000962_3.6551,NC000962_3.6553,NC000962_3.6571,NC000962_3.6575,NC000962_3.6579,NC000962_3.6586,NC000962_3.6620,NC000962_3.6638,NC000962_3.6695,NC000962_3.6735,NC000962_3.6738,NC000962_3.6742,NC000962_3.6749,NC000962_3.6750,NC000962_3.6807,NC000962_3.6878,NC000962_3.6881,NC000962_3.7058,NC000962_3.7088,NC000962_3.7170,NC000962_3.7268,NC000962_3.7355,NC000962_3.7362,NC000962_3.7496,NC000962_3.7563,NC000962_3.7564,NC000962_3.7566,NC000962_3.7567,NC000962_3.7570,NC000962_3.7572,NC000962_3.7581,NC000962_3.7582,NC000962_3.7585,NC000962_3.7607,NC000962_3.7631,NC000962_3.7637,NC000962_3.7652,NC000962_3.7658,NC000962_3.7664,NC000962_3.7683,NC000962_3.7685,NC000962_3.7694,NC000962_3.7710,NC000962_3.7712,NC000962_3.7725,NC000962_3.7728,NC000962_3.7730,NC000962_3.7892,NC000962_3.8040,NC000962_3.8164,NC000962_3.8201,NC000962_3.8434,NC000962_3.8452,NC000962_3.8519,NC000962_3.8619,NC000962_3.8624,NC000962_3.9023,NC000962_3.9032,NC000962_3.9034,NC000962_3.9050,NC000962_3.9051,NC000962_3.9113,NC000962_3.9119,NC000962_3.9134,NC000962_3.9143,NC000962_3.9145,NC000962_3.9147,NC000962_3.9153,NC000962_3.9154,NC000962_3.9155
SRR10525336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SRR10380004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SRR6807701,0,0,0,0,0,0,0

In [9]:

test = h2o.import_file( DATA_LOCATION + "processed/final.test.tsv")
test.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


SampleID,NC000962_3.22,NC000962_3.434,NC000962_3.524,NC000962_3.645,NC000962_3.648,NC000962_3.654,NC000962_3.666,NC000962_3.675,NC000962_3.678,NC000962_3.693,NC000962_3.698,NC000962_3.699,NC000962_3.702,NC000962_3.705,NC000962_3.708,NC000962_3.717,NC000962_3.729,NC000962_3.741,NC000962_3.744,NC000962_3.747,NC000962_3.750,NC000962_3.756,NC000962_3.770,NC000962_3.777,NC000962_3.780,NC000962_3.783,NC000962_3.793,NC000962_3.795,NC000962_3.799,NC000962_3.801,NC000962_3.805,NC000962_3.822,NC000962_3.840,NC000962_3.846,NC000962_3.849,NC000962_3.852,NC000962_3.1045,NC000962_3.1049,NC000962_3.1089,NC000962_3.1123,NC000962_3.1152,NC000962_3.1153,NC000962_3.1155,NC000962_3.1161,NC000962_3.1164,NC000962_3.1166,NC000962_3.1167,NC000962_3.1176,NC000962_3.1206,NC000962_3.1212,NC000962_3.1255,NC000962_3.1278,NC000962_3.1291,NC000962_3.1302,NC000962_3.1326,NC000962_3.1389,NC000962_3.1399,NC000962_3.1413,NC000962_3.1416,NC000962_3.1422,NC000962_3.1429,NC000962_3.1431,NC000962_3.1432,NC000962_3.1452,NC000962_3.1458,NC000962_3.1461,NC000962_3.1470,NC000962_3.1473,NC000962_3.1474,NC000962_3.1653,NC000962_3.1676,NC000962_3.1699,NC000962_3.1703,NC000962_3.1708,NC000962_3.1718,NC000962_3.1729,NC000962_3.1771,NC000962_3.1827,NC000962_3.1849,NC000962_3.1918,NC000962_3.1977,NC000962_3.2532,NC000962_3.2745,NC000962_3.3352,NC000962_3.3446,NC000962_3.4013,NC000962_3.4086,NC000962_3.4096,NC000962_3.4119,NC000962_3.4938,NC000962_3.5075,NC000962_3.5627,NC000962_3.5782,NC000962_3.5790,NC000962_3.5791,NC000962_3.5803,NC000962_3.5807,NC000962_3.5824,NC000962_3.5839,NC000962_3.5848,NC000962_3.5856,NC000962_3.5858,NC000962_3.5860,NC000962_3.5902,NC000962_3.6003,NC000962_3.6013,NC000962_3.6112,NC000962_3.6280,NC000962_3.6286,NC000962_3.6292,NC000962_3.6307,NC000962_3.6362,NC000962_3.6382,NC000962_3.6388,NC000962_3.6403,NC000962_3.6430,NC000962_3.6436,NC000962_3.6439,NC000962_3.6445,NC000962_3.6452,NC000962_3.6453,NC000962_3.6502,NC000962_3.6508,NC000962_3.6511,NC000962_3.6515,NC000962_3.6520,NC000962_3.6535,NC000962_3.6547,NC000962_3.6550,NC000962_3.6551,NC000962_3.6553,NC000962_3.6571,NC000962_3.6575,NC000962_3.6579,NC000962_3.6586,NC000962_3.6620,NC000962_3.6638,NC000962_3.6695,NC000962_3.6735,NC000962_3.6738,NC000962_3.6742,NC000962_3.6749,NC000962_3.6750,NC000962_3.6807,NC000962_3.6878,NC000962_3.6881,NC000962_3.7058,NC000962_3.7088,NC000962_3.7170,NC000962_3.7268,NC000962_3.7355,NC000962_3.7362,NC000962_3.7496,NC000962_3.7563,NC000962_3.7564,NC000962_3.7566,NC000962_3.7567,NC000962_3.7570,NC000962_3.7572,NC000962_3.7581,NC000962_3.7582,NC000962_3.7585,NC000962_3.7607,NC000962_3.7631,NC000962_3.7637,NC000962_3.7652,NC000962_3.7658,NC000962_3.7664,NC000962_3.7683,NC000962_3.7685,NC000962_3.7694,NC000962_3.7710,NC000962_3.7712,NC000962_3.7725,NC000962_3.7728,NC000962_3.7730,NC000962_3.7892,NC000962_3.8040,NC000962_3.8164,NC000962_3.8201,NC000962_3.8434,NC000962_3.8452,NC000962_3.8519,NC000962_3.8619,NC000962_3.8624,NC000962_3.9023,NC000962_3.9032,NC000962_3.9034,NC000962_3.9050,NC000962_3.9051,NC000962_3.9113,NC000962_3.9119,NC000962_3.9134,NC000962_3.9143,NC000962_3.9145,NC000962_3.9147,NC000962_3.9153,NC000962_3.9154,NC000962_3.9155
ERR3335735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
SRR8552929,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0
ERR067629,0,0,0,0,0,0,0,0,

In [10]:
# Identify predictors and response
train_predictor_cols = train.columns
train_response_col = "Resistance_Status"
train_predictor_cols.remove('SampleID')
train_predictor_cols.remove(train_response_col)
print("train frame - predictor column: ", train_predictor_cols[0], train_predictor_cols[-1])
print("train frame - response column: ", train_response_col)



# Identify predictors and response
test_predictor_cols = test.columns
test_response_col = "Resistance_Status"
test_predictor_cols.remove('SampleID')
test_predictor_cols.remove(test_response_col)
print("test frame - predictor columns: ", test_predictor_cols[0], test_predictor_cols[-1])
print("test frame - response column: ", test_response_col)

train frame - predictor column:  NC000962_3.22 NC000962_3.4411327
train frame - response column:  Resistance_Status
test frame - predictor columns:  NC000962_3.22 NC000962_3.4411327
test frame - response column:  Resistance_Status


In [11]:

# For binary classification, response should be a factor
train[train_response_col] = train[train_response_col].asfactor()
test[test_response_col] = test[test_response_col].asfactor()


# Number of CV folds (to generate level-one data for stacking)
nfolds = 5

MAX_GRID_MODELS = 10


x = train_predictor_cols
y = train_response_col




# Stacked ensemble from grid models

In [13]:
nb_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/nb_grid/Grid_NaiveBayes_py_3_sid_9644_model_python_1604419067081_1")



glm_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/glm_grid/Grid_GLM_py_3_sid_b7a1_model_python_1604419221083_1")



gbm_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/gbm_grid/Grid_GBM_py_7_sid_9651_model_python_1604407520638_1")



xgb_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/xgb_grid/Grid_XGBoost_py_7_sid_a3b5_model_python_1604427337744_1")



dl_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/dl_grid/Grid_DeepLearning_py_3_sid_b7a1_model_python_1604419221083_608")


drf_grid = h2o.load_grid(MODELS_LOCATION + "FINAL/drf_grid/Grid_DRF_py_3_sid_9421_model_python_1604478808297_199")


In [18]:
# Select best models from the grids based on performance on the test data

def best_model_from_grid (model_grid):
    best_model = model_grid[0]
    for mdl in model_grid:
        if (mdl.model_performance(test).auc() > best_model.model_performance(test).auc()):
            best_model = mdl
    print("Modeld ID: ", model_grid.grid_id ,"\nAUC of best model on training data: ", best_model.auc(),
                                             "\nAUC of best model on validation data: ", best_model.model_performance(test).auc(), "\n@@@@@@@@@@@@@@@@\n")
    return best_model

In [19]:
best_nb_model = best_model_from_grid(nb_grid)
best_glm_model = best_model_from_grid(glm_grid)
best_gbm_model = best_model_from_grid(gbm_grid)
best_xgb_model= best_model_from_grid(xgb_grid)
best_dl_model= best_model_from_grid(dl_grid)
best_drf_model= best_model_from_grid(drf_grid)

Modeld ID:  Grid_NaiveBayes_py_3_sid_9644_model_python_1604419067081_1 
AUC of best model on training data:  0.751100768321513 
AUC of best model on validation data:  0.6308581696736486 
@@@@@@@@@@@@@@@@

Modeld ID:  Grid_GLM_py_3_sid_b7a1_model_python_1604419221083_1 
AUC of best model on training data:  0.9639509456264777 
AUC of best model on validation data:  0.7338639416270374 
@@@@@@@@@@@@@@@@

Modeld ID:  Grid_GBM_py_7_sid_9651_model_python_1604407520638_1 
AUC of best model on training data:  0.9466164302600473 
AUC of best model on validation data:  0.684212436925981 
@@@@@@@@@@@@@@@@

Modeld ID:  Grid_XGBoost_py_7_sid_a3b5_model_python_1604427337744_1 
AUC of best model on training data:  0.9693942080378251 
AUC of best model on validation data:  0.7161487639307366 
@@@@@@@@@@@@@@@@

Modeld ID:  Grid_DeepLearning_py_3_sid_b7a1_model_python_1604419221083_608 
AUC of best model on training data:  0.995475768321513 
AUC of best model on validation data:  0.637927905035031 
@@@@@

In [25]:

def extract_params_from_model(actual_params_dict, extra_params = [], additional_keys = {}):
    final_params = actual_params_dict

    columns_to_be_removed =   [
                                'model_id',
                                'validation_frame',
                                'response_column',
                                'ignored_columns',
                                'training_frame',
                                *extra_params
]

    for col_name in columns_to_be_removed:
        del  final_params[col_name]

    return {**final_params, **additional_keys}

In [ ]:
from h2o.estimators import H2ONaiveBayesEstimator

top_nb = H2ONaiveBayesEstimator(**extract_params_from_model(best_nb_model.actual_params))

top_nb.train(x=x, y=y, training_frame=train, validation_frame=test)

print('AUC on test data: ', top_nb.model_performance(test).auc(), "\n\n============================")

top_nb.model_performance

In [ ]:
from h2o.estimators import H2OGeneralizedLinearEstimator

top_glm = H2OGeneralizedLinearEstimator(**extract_params_from_model(best_glm_model.actual_params, ['lambda']))

top_glm.train(x=x, y=y, training_frame=train, validation_frame=test)

print('AUC on test data: ', top_glm.model_performance(test).auc(), "\n\n============================")

top_glm.model_performance

In [ ]:
from h2o.estimators import H2OGradientBoostingEstimator

top_gbm = H2OGradientBoostingEstimator(**extract_params_from_model(best_gbm_model.actual_params))

top_gbm.train(x=x, y=y, training_frame=train, validation_frame=test)

print('AUC on test data: ', top_gbm.model_performance(test).auc(), "\n\n============================")

top_gbm.model_performance

In [ ]:
from h2o.estimators import H2OXGBoostEstimator

top_xgb = H2OXGBoostEstimator(**extract_params_from_model(best_xgb_model.actual_params))

top_xgb.train(x=x, y=y, training_frame=train, validation_frame=test)

print('AUC on test data: ', top_xgb.model_performance(test).auc(), "\n\n============================")

top_xgb.model_performance

In [ ]:
from h2o.estimators import H2ODeepLearningEstimator

top_dl = H2ODeepLearningEstimator(**extract_params_from_model(best_dl_model.actual_params))

top_dl.train(x=x, y=y, training_frame=train, validation_frame=test)

print('AUC on test data: ', top_dl.model_performance(test).auc(), "\n\n============================")

top_dl.model_performance

In [ ]:
from h2o.estimators import H2ORandomForestEstimator

top_drf = H2ORandomForestEstimator(extract_params_from_model(best_drf_model.actual_params, 
                                                                extra_params=['weights_column'],
                                                                additional_keys= {'nfolds':5,
                                                                                  'fold_assignment':'random'}))


top_drf.train(x=x, y=y, training_frame=train, validation_frame=test)

print('AUC on test data: ', top_drf.model_performance(test).auc(), "\n\n============================")

top_drf.model_performance

In [ ]:
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator


ensemble = H2OStackedEnsembleEstimator(
                                       base_models= [
                                                    top_nb,
                                                    top_glm,

                                                    # models with checkpoint available
                                                    top_gbm,
                                                    top_xgb,
                                                    top_dl,
                                                    top_drf
                                                    ]
                                       )

ensemble.train(x=x, y=y, training_frame=train, validation_frame=test)

print('AUC on test data: ', ensemble.model_performance(test).auc(), "\n\n============================")
ensemble.model_performance

### Check for the best  meta-learner

In [ ]:
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

collection_of_models = [top_nb,
                        top_glm,

                        # checkpoint-enabled models
                        top_gbm,
                        top_xgb,
                        top_dl,
                        top_drf]


meta_algos = ["auto", "xgboost", "drf", "gbm", "glm", "naivebayes", "deeplearning"]

ensemble_list = []

for metalearner in meta_algos:
    print("\n\n>>>>> ", metalearner, " <<<<<<")

    ensemble = H2OStackedEnsembleEstimator(
                                       base_models= collection_of_models,

                                       model_id= "stacked_ensemble_metalearner_" + metalearner,

                                       metalearner_algorithm= metalearner,

                                       #metalearner_params
                                       )
    ensemble.train(x=x, y=y, training_frame=train, validation_frame=test)
    print("AUC on test data: ",  ensemble.model_performance(test).auc())

    ensemble_list.append(ensemble)




